In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import panel as pn
pn.extension("ipywidgets", 'katex', 'mathjax')
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

from random import shuffle, uniform

print("Packages succesfully loaded")

# From week 2

some widgets have been removed from week 2

In [ ]:
def W2_Q8():

    # define widgets
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description='a [m]')
    a2 = ipw.FloatText(value=0.5, min=0, max=20, step=0.01, description='a [m]')

    T1 = ipw.FloatText(value=8, min=0.01, max=250000, step=0.01, description='T [s]')
    T2 = ipw.FloatText(value=4, min=0.01, max=250000, step=0.01, description='T [s]')

    phi_1 = ipw.FloatText(value=0, min=-1, max=1, step=0.01, description='phi [2 pi rad]')#'$\phi$ [2 $\pi$ rad]')
    phi_2 = ipw.FloatText(value=0.25, min=-1, max=1, step=0.01, description='phi [2 pi rad]')#'$\phi$ [2 $\pi$ rad]')
    
    n_waves = ipw.FloatText(value=3, min=0.1, max=10, step=0.1, description='n_{waves}')
    
    depth = ipw.FloatText(value=7.5, min=1, max=250, step=0.01, description='h [m]')
    xp = ipw.FloatText(value=0, min=0, step=0.1, description='x [m]')
    tp = ipw.FloatText(value=0, min=1, step=0.1, description='t [s]')
    
    # Disabled/informative widgets
    L1 = ipw.FloatText(value=wave_length(T1.value,depth.value), description='L [m]', disabled=True)
    L2 = ipw.FloatText(value=wave_length(T2.value,depth.value), description='L [m]', disabled=True)
    Lgroup = ipw.FloatText(value=group_stats(k1=2 * np.pi / L1.value, k2=2 * np.pi / L2.value, w1=2 * np.pi / T1.value, w2=2 * np.pi / T2.value)[0], description='L_{group} [m]', disabled=True)

    # Setup widget layout (User Interface) and display
    vbox1 = ipw.VBox([ipw.Label('Wave 1', layout=ipw.Layout(align_self='center')),a1, T1, phi_1, L1])
    vbox2 = ipw.VBox([ipw.Label('Wave 2', layout=ipw.Layout(align_self='center')),a2, T2, phi_2, L2])
    vbox3 = ipw.VBox([ipw.Label('Wave group', layout=ipw.Layout(align_self='center')), n_waves, Lgroup])
    vbox4 = ipw.VBox([ipw.Label('General', layout=ipw.Layout(align_self='center')),depth, xp, tp])
    
    ui = ipw.HBox([vbox1, vbox2, vbox3, vbox4])

    def calc_eta(a1,T1,phi_1, a2,T2, phi_2, L1,L2, n_waves,xp, tp, depth):
        L1 = wave_length(T1,depth)
        L2 = wave_length(T2,depth)
        
        L_group, T_group, c_g = group_stats(
        k1=2 * np.pi / L1,
        k2=2 * np.pi / L2,
        w1=2 * np.pi / T1,
        w2=2 * np.pi / T2,
        )
        
        T = np.min([T1,T2])
        L = np.max([L1, L2])
        t = np.arange(0,n_waves*T_group+T/30,T/30)
        x = np.arange(0,n_waves*L_group+L/30,L/30)
        
        fig, axs = plt.subplots(nrows = 3,ncols = 2,figsize = (9,5), sharex=False, sharey = False)
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)
        
        # time based
        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[2,0]
        #space based
        ax4 = axs[0,1]
        ax5 = axs[1,1]
        ax6 = axs[2,1]

        # calculate surface, including phase change
        eta1_T = a1*np.sin(2*np.pi/T1*t-2*np.pi/L1*xp-phi_1*(2*np.pi))
        eta2_T = a2*np.sin(2*np.pi/T2*t-2*np.pi/L2*xp-phi_2*(2*np.pi))
        eta_T = eta1_T + eta2_T

        eta1_x= a1*np.sin(2*np.pi/T1*tp-2*np.pi/L1*x-phi_1*(2*np.pi))
        eta2_x= a2*np.sin(2*np.pi/T2*tp-2*np.pi/L2*x-phi_2*(2*np.pi))
        eta_x = eta1_x + eta2_x

        # calculate surface, without phase change
        eta1_T_basic = a1*np.sin(2*np.pi/T1*t-2*np.pi/L1*xp)
        eta2_T_basic = a2*np.sin(2*np.pi/T2*t-2*np.pi/L2*xp)
        eta_T_basic = eta1_T_basic + eta2_T_basic

        eta1_x_basic = a1*np.sin(2*np.pi/T1*tp-2*np.pi/L1*x)
        eta2_x_basic = a2*np.sin(2*np.pi/T2*tp-2*np.pi/L2*x)
        eta_x_basic = eta1_x_basic + eta2_x_basic

        # plot surface excluding phase change
        ax1.plot(t,eta1_T_basic, color = 'grey', linestyle = '--', label = 'reference')
        ax2.plot(t,eta2_T_basic, color = 'grey', linestyle = '--', label = 'reference')
        ax3.plot(t, eta_T_basic, color = 'grey', linestyle = '--', label = '$\eta_1$')
        ax4.plot(x, eta1_x_basic, color = 'grey', linestyle = '--', label = 'reference')
        ax5.plot(x, eta2_x_basic, color = 'grey', linestyle = '--', label = 'reference')
        ax6.plot(x, eta_x_basic, color = 'grey', linestyle = '--', label = '$\eta_1$')
        
        # plot surface including phase change
        ax1.plot(t,eta1_T, label = '$\eta_1$')
        ax2.plot(t,eta2_T, label = '$\eta_2$' )
        ax3.plot(t,eta_T, label = '$\eta_{1+2}$')
        
        ax4.plot(x,eta1_x, label = '$\eta_1$')
        ax5.plot(x,eta2_x, label = '$\eta_2$')
        ax6.plot(x,eta_x, label = '$\eta_{1+2}$')

        # set vertical axis the same
        amp = (a1+a2)*1.1
        ax1.set_ylim(-amp,amp)
        ax2.set_ylim(-amp,amp)
        ax3.set_ylim(-amp,amp)
        ax4.set_ylim(-amp,amp)
        ax5.set_ylim(-amp,amp)
        ax6.set_ylim(-amp,amp)

        # set horizontal axis
        ax1.set_xlim(0,n_waves*T_group)
        ax2.set_xlim(0,n_waves*T_group)
        ax3.set_xlim(0,n_waves*T_group)
        ax4.set_xlim(0,n_waves*L_group)
        ax5.set_xlim(0,n_waves*L_group)
        ax6.set_xlim(0,n_waves*L_group)

        # set labels
        ax1.set_ylabel('$\eta_1$ [m]')
        ax2.set_ylabel('$\eta_2$ [m]')
        ax3.set_ylabel('$\eta_{1+2}$ [m]')
        
        ax3.set_xlabel('t/T_{group}')
        ax6.set_xlabel('x/L_{group}')

        # remove the lines related to the x-ticks and y-ticks
        ax1.xaxis.set_visible(False)
        ax2.xaxis.set_visible(False)
        ax4.yaxis.set_visible(False)
        ax5.yaxis.set_visible(False)
        ax6.yaxis.set_visible(False)
        
        # set scaled ticks
        if n_waves >= 1:
            ax3.set_xticks(np.arange(0,n_waves//1 +1, 1)*T_group)
            ax3.set_xticklabels(np.arange(0,n_waves//1 +1, 1))
            ax6.set_xticks(np.arange(0,n_waves//1 +1, 1)*L_group)
            ax6.set_xticklabels(np.arange(0,n_waves//1 +1, 1))

        else: # 3 times when the scale is smaller than 1
            ax3.set_xticks([0,0.5*n_waves*T_group, n_waves*T_group])
            ax3.set_xticklabels([0, 0.5*n_waves, n_waves])
            ax6.set_xticks([0,0.5*n_waves*L_group, n_waves*L_group])
            ax6.set_xticklabels([0, 0.5*n_waves, n_waves])
        
        # remove x and y ticks
        ax4.set_xticklabels([], fontsize=0)
        ax5.set_xticklabels([], fontsize=0)
        
        ax4.set_yticklabels([], fontsize=0)
        ax5.set_yticklabels([], fontsize=0)
        ax6.set_yticklabels([], fontsize=0)
        
        # set title
        ax1.set_title('Time-based (x =' + str(xp) + ' m)')
        ax4.set_title('Space-based (t =' + str(tp) + ' s)')

        # plot legends       
        legend1 = ax4.legend(loc='center left', bbox_to_anchor=(1.001, 0.5))
        legend2 = ax5.legend(loc='center left', bbox_to_anchor=(1.001, 0.5))
        legend3 = ax6.legend(loc='center left', bbox_to_anchor=(1.001, 0.5))

        ui = ipw.HBox([vbox1, vbox2, vbox3, vbox4])

    #update graph
    out = ipw.interactive_output(calc_eta, {'a1': a1,'T1':T1, 'phi_1': phi_1, 'a2': a2,'T2':T2, 'phi_2': phi_2, 'L1' : L1, 'L2': L2, 'n_waves': n_waves, 'xp':xp, 'tp': tp, 'depth':depth})

    display(ui,out)

    

W2_Q8()